In [11]:
# ==============================================================
#  ASISTENTE INTELIGENTE DE EVALUACIÓN DE DIFICULTADES
#  Proyecto SIC 2025 - Modelo de Clasificación con Random Forest
# ==============================================================
# --------------------------------------------------------------
# IMPORTS
# --------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# --------------------------------------------------------------
# DATASET AUTOMÁTICO
# --------------------------------------------------------------
np.random.seed(42)

n = 300  # número de estudiantes simulados

df = pd.DataFrame({
    "tiempo_promedio_respuesta": np.random.normal(18, 3, n),
    "tasa_error": np.random.uniform(0, 1, n),
    "intentos_fallidos": np.random.randint(0, 10, n),
    "varianza_respuestas": np.random.normal(1.5, 0.6, n),
    "consistencia": np.random.uniform(0, 1, n),
})

# Reglas simples para generar la etiqueta
def asignar_riesgo(row):
    score = 0
    score += row["tasa_error"] * 0.5
    score += (row["intentos_fallidos"] / 10) * 0.3
    score += (row["tiempo_promedio_respuesta"] / 30) * 0.2

    if score < 0.33:
        return "Bajo"
    elif score < 0.66:
        return "Medio"
    else:
        return "Alto"

df["riesgo"] = df.apply(asignar_riesgo, axis=1)

print("Vista previa del dataset:")
display(df.head())

# --------------------------------------------------------------
# SEPARAR VARIABLES
# --------------------------------------------------------------
X = df.drop("riesgo", axis=1)
y = df["riesgo"]

# --------------------------------------------------------------
# DIVISIÓN DE DATOS
# --------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# --------------------------------------------------------------
# ESCALADO
# --------------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --------------------------------------------------------------
# ENTRENAR RANDOM FOREST
# --------------------------------------------------------------
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42
)

model.fit(X_train_scaled, y_train)

# --------------------------------------------------------------
# EVALUACIÓN DEL MODELO
# --------------------------------------------------------------
y_pred = model.predict(X_test_scaled)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

# --------------------------------------------------------------
# DEMOSTRACIÓN: PREDICCIÓN DE UN ESTUDIANTE NUEVO
# --------------------------------------------------------------
estudiante_nuevo = pd.DataFrame({
    "tiempo_promedio_respuesta": [17],
    "tasa_error": [0.42],
    "intentos_fallidos": [5],
    "varianza_respuestas": [1.3],
    "consistencia": [0.55]
})

estudiante_scaled = scaler.transform(estudiante_nuevo)
prediccion = model.predict(estudiante_scaled)

print("\nPredicción para estudiante nuevo:", prediccion[0])


Vista previa del dataset:


,tiempo_promedio_respuesta,tasa_error,intentos_fallidos,varianza_respuestas,consistencia,riesgo
0,19.490142,0.282035,7,0.447918,0.643578,Medio
1,17.585207,0.177440,9,1.173047,0.526403,Medio
2,19.943066,0.750615,2,2.083094,0.731590,Medio
3,22.569090,0.806835,4,1.481425,0.081630,Alto
4,17.297540,0.990505,5,1.298571,0.060352,Alto



Accuracy: 0.8533333333333334

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       0.92      0.75      0.83        16
        Bajo       1.00      0.54      0.70        13
       Medio       0.82      0.98      0.89        46

    accuracy                           0.85        75
   macro avg       0.91      0.76      0.81        75
weighted avg       0.87      0.85      0.84        75


Predicción para estudiante nuevo: Medio
